## __Chapter 01: Untitled__
---
by Daniel Weller & Holger Mann

In [ ]:
data.packages <- c("bigmemory", "data.table", "rgdal", "rgeos")
lapply(data.packages, require, character.only=T); rm(data.packages)

stat.packages <- c("bivariate", "extRemes", "ks", "mev", "mvtnorm")
lapply(stat.packages, require, character.only=T); rm(stat.packages)

if(.Platform$OS.type!="windows"){
  paths <- c("/Volumes/Climate Rasters/", "/Volumes/Analysis Storage", "~/GitHub/Report/")} else {
    paths <- c("D:/Climate Rasters/", "E:/Analysis Storage/", "C:/Users/Cloud/GitHub/Report/")}
setwd(paths[3]); options(scipen=10); set.seed(12345); setDTthreads(16)
rasterOptions(progress="text", maxmemory=1e+09, chunksize=1e+08)

In [ ]:
xaxis <- 721; yaxis <- 1440; years <- 41; hours <- 24; cells <- xaxis*yaxis; temps <- 366*years
min <- big.matrix(nrow=cells, ncol=temps, init=NULL, separated=FALSE, backingpath=paths[2],
                  backingfile="Min_Temp_Back", descriptorfile="Min_Temp_Desc")
max <- big.matrix(nrow=cells, ncol=temps, init=NULL, separated=FALSE, backingpath=paths[2],
                  backingfile="Max_Temp_Back", descriptorfile="Max_Temp_Desc")

for(j in 1:length(list.files(paths[1]))){
  grib <- readGDAL(paste(paths[1], list.files(paths[1])[j], sep=""))
  if(j==1){count <- 1}
  for(i in seq(1, ncol(grib@data), by=hours)){
    min[,count] <- apply(grib@data[,c(i:(i+hours-1))]-273.15, 1, min)
    max[,count] <- apply(grib@data[,c(i:(i+hours-1))]-273.15, 1, max)
    if((ncol(grib@data)==672) & ((i+hours-1)==672)){
      min[,count+1] <- rep(NA, cells)
      max[,count+1] <- rep(NA, cells)
      count <- count+2} else {count <- count+1}}
  rm(grib); invisible(gc())
  if(j %in% seq(12, years*12, by=12)){Sys.sleep(60)}}
rm(xaxis, yaxis, temps, hours, count); invisible(gc())

In [ ]:
flex.wave <- function(x, yrs.start=NULL, yrs.end=NULL, mth.start=NULL, mth.end=NULL, thresh=NULL,
                      type=c("heat","cold"), flex.days=15, flex.yrs=15, flex.prob=0.90, wave.span=3){
  if(is.matrix(x) | !is.vector(x)){stop("\nIncorrect input: 'x' should be a vector.")}
  if(is.numeric(c(yrs.start, yrs.end, mth.start, mth.end))==FALSE){
    stop("\nIncorrect input: All temporal parameters should be numeric.")} else {
      if(length(yrs.start)==0L){
        stop("\nMissing input: 'yrs.start' hasn't been set. Please set 'yrs.end' as well.")} else
          if(length(yrs.end)==0L){yrs.end <- yrs.start}
      if(all(length(mth.start)==0L, length(mth.end)==0L)){
        mth.start <- 1L
        mth.end <- 12L}}

  dpm <- c(31L, 29L, 31L, 30L, 31L, 30L, 31L, 31L, 30L, 31L, 30L, 31L)
  all.days <- list(1:dpm[1], 1:dpm[2], 1:dpm[3], 1:dpm[4], 1:dpm[5], 1:dpm[6],
                   1:dpm[7], 1:dpm[8], 1:dpm[9], 1:dpm[10], 1:dpm[11], 1:dpm[12])
  all.years <- yrs.start:yrs.end
  n.years <- length(all.years)

  if(n.years==1){
    all.dpm <- dpm[mth.start:mth.end]
    n.days <- sum(all.dpm)
    if(n.days==length(x)){
      year <- rep(yrs.start, length(x))
      month <- rep(mth.start:mth.end, dpm[mth.start:mth.end])
      day <- unlist(all.days[mth.start:mth.end])} else {
        stop("\nFatal error: 'x' should have ", n.days, " days, but has ", length(x), "!")}} else {
          all.dpm <- c(dpm[mth.start:12], rep(dpm, (n.years-2)), dpm[1:mth.end])
          n.days <- sum(all.dpm)
          if(n.days==length(x)){
            year <- rep(all.years, c(sum(dpm[mth.start:12]), rep(366, length(all.years)-2), 
                                     sum(dpm[1:mth.end])))
            month <- c(rep(mth.start:12, dpm[mth.start:12]),
                       rep(rep(1:12, length(all.years)-2), rep(dpm, length(all.years)-2)),
                       rep(1:mth.end, dpm[1:mth.end]))
            days <- c(unlist(all.days[mth.start:12]), rep(unlist(all.days), length(all.years)-2),
                      unlist(all.days[1:mth.end]))} else {
                        stop("\nFatal error: 'x' should have ", n.days, " days, but has ", length(x), "!")}}

  ymd <- cbind(year, month, days)

  if(length(thresh)==0){
    stop("\nMissing input: 'thresh' hasn't been set. Please provide a value or specify 'flex'!")}

  if(thresh=="flex"){
    flx.stats <- vector()
    for(j in seq(1, n.years*366, by=1)){
      flx.frame <- seq(j-flex.days, j+flex.days, by=1)
      flx.dates <- sort(outer(flx.frame, seq(-flex.yrs, flex.yrs)*366, "+"))
      flx.value <- x[flx.dates[(flx.dates >= 1) & (flx.dates <= n.years*366)]]
      flx.stats[j] <- quantile(flx.value, probs=flex.prob, na.rm=TRUE)[[1]]}
    thresh <- round(flx.stats, digits=2)} else {thresh <- round(rep(thresh, length(x)), digits=2)}

  if(type=="cold"){
    x <- -1*x
    if(isFALSE(all(thresh==thresh[1]) & thresh[1] > 0)){
      thresh <- -1*thresh}}

  days.count <- vector(mode="integer", length=length(x))
  max.temp <- vector(mode="integer", length=length(x))
  wid <- vector(mode="integer", length=length(x))

  for(i in 1:length(x)){
    if(!is.numeric(x[i]) | is.na(x[i])){
      if(i==1){
        stop("\nFatal error: Missing value for first element of 'x' found.")}
      days.count[i] <- NA
      max.temp[i] <- NA
      wid[i] <- NA} else if(i==1){
        days.count[i] <- hlp.count <- 0
        max.temp[i] <- hlp.temp <- NA
        wid[i] <- hlp.id <- 0
        if(x[i] > thresh[i]){
          days.count[i] <- hlp.count <- 1
          max.temp[i] <- hlp.temp <- x[i]
          wid[i] <- hlp.id <- 0
          buster <- 0
          if(length(x[i:(i+wave.span-1)][!is.na(x[i:(i+wave.span-1)])]) < wave.span){
            buster <- 1}
          if(all((x[i:(i+wave.span+buster-1)] > thresh[i:(i+wave.span+buster-1)]), na.rm=TRUE)){
            hlp.id <- hlp.id+1
            wid[i] <- hlp.id}}} else {
              if(x[i] > thresh[i]){
                hlp.count <- hlp.count+1
                days.count[i] <- hlp.count
                if(is.na(hlp.temp) | hlp.temp < x[i]){hlp.temp <- x[i]}
                max.temp[i] <- hlp.temp
                buster <- 0
                if(length(x[i:(i+wave.span-1)][!is.na(x[i:(i+wave.span-1)])]) < wave.span){
                  buster <- 1}
                if(days.count[i]==1 & all((x[i:(i+wave.span+buster-1)] > 
                                           thresh[i:(i+wave.span+buster-1)]), na.rm=TRUE)){
                  hlp.id <- hlp.id+1
                  wid[i] <- hlp.id} else {
                    wid[i] <- hlp.id
                    if((days.count[i] %in% 1:(wave.span-1)) & 
                       any(x[(i+1):(i+1+wave.span-days.count[i]+buster-1)] <= 
                           thresh[(i+1):(i+1+wave.span-days.count[i]+buster-1)], na.rm=TRUE)){
                      wid[i] <- 0}}} else {
                        days.count[i] <- hlp.count <- 0
                        max.temp[i] <- hlp.temp <- NA
                        wid[i] <- 0}}}

  if(type=="cold"){
    x <- -1*x
    thresh <- -1*thresh
    max.temp <- -1*max.temp}

  pre.erg <- as.data.table(cbind(ymd, x, days.count, max.temp, wid, thresh))
  fin.erg <- as.data.table(lapply(pre.erg, function(x){
    if(all(x[!is.na(x)]%%1==0)==TRUE) {as.integer(x)} else {x}}))
  names(fin.erg) <- c("Year", "Month", "Day", "MaxDay", "DaysCount", "MaxWave", "WaveID", "Thresh")
  return(fin.erg)}

In [ ]:
cell.list <- vector()
for(i in 1:cells){
  if(all(!is.na(min[i,])) & all(!is.na(max[i,]))){
    cold <- flex.wave(min[i,], yrs.start=1979, yrs.end=2019, type="cold", thresh="flex", flex.prob=0.10)
    fwrite(cold, paste(paths[2], "Cold Wave Cell ", i, ".csv", sep=""))
    heat <- flex.wave(max[i,], yrs.start=1979, yrs.end=2019, type="heat", thresh="flex", flex.prob=0.90)
    fwrite(heat, paste(paths[2], "Heat Wave Cell ", i, ".csv", sep=""))
    cell.list[i] <- i} else {cell.list[i] <- NA}}
rm(min, max, heat, cold, cells); invisible(gc())
cell.list <- na.omit(cell.list)